# Restricted Boltzmann Machine
grouplens.org/datasets/movielens

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data Preprocessing

In [3]:
# importing the dataset
# movie id, movie name, movie genre
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header = None, engine = 'python', encoding='latin-1')
movies.head(10)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
# user id, gender, age, occupation code, zip code
users = pd.read_csv('ml-1m/users.dat', sep='::', header = None, engine = 'python', encoding='latin-1')
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [5]:
# user id, movie id, ratings, time stamp when the movie was rated
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header = None, engine = 'python', encoding='latin-1')
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [6]:
# Preparing the training set and the test set 
training_set = pd.read_csv('ml-100k/u1.base', header = None, delimiter = '\t')
training_set = np.array(training_set, dtype='int')
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ..., 
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [7]:
test_set = pd.read_csv('ml-100k/u1.test', header = None, delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ..., 
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [8]:
# getting the number of users and movies
nb_users = max(max(training_set[:,0]),max(test_set[:,0]))
nb_movies = max(max(training_set[:,1]),max(test_set[:,1]))
nb_users
nb_movies

943

1682

In [9]:
# converting the data into an array (list of lists) with users in lines and movies in columns
# movie 1 with ratings [[1,0,0,5,.... ]]
# movie 2 with ratings [[2,0,0,3,4,5,...]]
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [10]:
training_set = convert(training_set)
test_set = convert(test_set)


In [11]:
def peek(data):
    for i in range(0,10):
        print data[i]
peek(training_set)

[5.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 4.0, 0.0,

In [12]:
# tensors are multi-d matrix (pytorch array) with with elements of single data type
# convert data into torch sensors
training_set =  torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [13]:
# convert the ratings into binary ratings 1 (liked) or 0 (not liked)
training_set[training_set == 0] = -1 # movies with non existant ratings
training_set[training_set == 1] = 0 # movies with bad ratings
training_set[training_set == 2] = 0 # movies with bad ratings
training_set[training_set >= 3] = 1 # movies with 3,4,5 stars have rating 1

In [14]:
test_set[test_set == 0] = -1 # movies with non existant ratings
test_set[test_set == 1] = 0 # movies with bad ratings
test_set[test_set == 2] = 0 # movies with bad ratings
test_set[test_set >= 3] = 1 # movies with 3,4,5 stars have rating 1

In [15]:
peek(training_set)


 1
 1
 1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


 1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


 1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]


-1
-1
-1
⋮ 
-1
-1
-1
[torch.FloatTensor of size 1682]



In [16]:
# creating the architecture of neural network 
class RBM(): 
    # nv = number of visible nodes
    # nh = number of hidden nodes 
    def __init__(self, nv, nh):
        # initialize the weight and bias 
        # weight W initialized as torch tensor 
        self.W = torch.randn(nh, nv)
        # one bias for each hidden node and edge hidden node, create vector of edge elements initialized by normal distribution
        # one dimension for batch and one dimension for bias
        self.a = torch.randn(1, nh)
        # for visible nodes now 
        self.b = torch.randn(1, nv)
        
    # Gibbs sampling to approximate log likelyhood gradients 
    # compute probabilities of hidden nodes given visible nodes 
    # with probabilities we can sample activations 
    def sample_h(self, x):
        # self to access variables 
        # x is the visible neurons v in the probabilities ph given v 
        
        #first compute probability of h given v <- probability that the hidden neuron is activated (1) given the values of divisible neurons
        wx = torch.mm(x, self.W.t()) # product of two torch tensors (remember this is two matricies)
        activation = wx + self.a.expand_as(wx) # bias of hidden nodes applied to each line of mini batch by using expand_as
        
        # Ex: user that likes only drama genre
        # Probability of hidden node with drama features with vislbe nodes => all drama nodes are 1 
        # sigmoid activation linear combination of neurons 
        p_h_given_v = torch.sigmoid(activation) # probabily of hidden node given value of visible node 
        
        # return probability
        # return sample of age sample of all hidden nodes and neurons according to probability p_h
        # if random number > p_h_given_v will not activate neuron, else activate neuron 
        return p_h_given_v, torch.bernoulli(p_h_given_v)
           
    def sample_v(self, y):
        wy = torch.mm(y, self.W) # product of two torch tensors (remember this is two matricies)
        activation = wy + self.b.expand_as(wy) # bias of hidden nodes applied to each line of mini batch by using expand_as
        p_v_given_h = torch.sigmoid(activation) # probabily of visible node given value of hidden node 
        
        # if random number > p_h_given_v will not activate neuron, else activate neuron 
        # visible node will get 0 -> not liked 
        return p_v_given_h, torch.bernoulli(p_v_given_h)
        
    # update our tensor of weights w, bias b and bias a
    # input vector v0 containing ratings of all movies by one user 
    # vk - visible nodes after k samplings 
    # ph0 - vector of probabilities that at first iteration hidden nodes equal 1 given values of v0 
    # phk - probabilities of hidden nodes after k sampling given the values of visible nodes vk 
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0) # retain format of tensor in 2-D
        self.a += torch.sum((ph0 - phk), 0)
        

In [17]:
nv = len(training_set[0])
nh = 100 # detecting 100 features 
batch_size = 100

In [18]:
rbm = RBM(nv, nh)

In [19]:
# training the rbm 
nb_epoch = 10 
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    # update weight after each batch of users going through the network 
    # first batch 0-99, second batch 100,199, etc. 
    for id_user in range(0,nb_users - batch_size, batch_size): 
        vk = training_set[id_user:id_user+batch_size] # vk ratings in the batch that already existed 
        v0 = training_set[id_user:id_user+batch_size] # ratings of movies that are already rated by the batch 
        ph0,_ =  rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0,vk,ph0,phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[vk>=0])) # real original ratings - prediction ratings v0>=0 means existant ratings
        s += 1. # decimal for float
    print "epoch: {} loss: {}".format(str(epoch), str(train_loss/s))

/anaconda2/lib/python2.7/site-packages/torch/tensor.py:309: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)


epoch: 1 loss: 0.293406143151
epoch: 2 loss: 0.251125110764
epoch: 3 loss: 0.250514112271
epoch: 4 loss: 0.251415547921
epoch: 5 loss: 0.250258494391
epoch: 6 loss: 0.250355428171
epoch: 7 loss: 0.252720429686
epoch: 8 loss: 0.249135552312
epoch: 9 loss: 0.252698594156
epoch: 10 loss: 0.249441303488


In [20]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print "test loss: {}".format(str(test_loss/s))

test loss: 0.239291134849
